In [2]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
import time

In [14]:
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("/Users/minjikim/Downloads/chromedriver 4")

# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

In [15]:
# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("Rome gelato")

# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [1]:
#1단계 검색어로 매장이름 데이터 만들기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

In [6]:
#selenium에서 사용할 모듈 import
import time
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [8]:
def search_bulit():
    """
    구글맵 검색어 정의(지역 + 코드 + restaurants)
    """
    df = pd.read_csv('romegelato.csv', encoding='utf-8')
    food_city = df['cafe_name']
    food_code = df['zip']
    food_state = df['state']
    search_name = []
    for i in range(len(food_city)):
            name = food_city[i] + " " + str(food_code[i]) + " Restaurants" 
            search_name.append(name)
    return search_name, food_code

In [9]:
def wait_input(driver):
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tactile-searchbox-input"))
        ) #입력창이 뜰 때까지 대기
    finally:
        pass

In [10]:
def input_funtion(n,driver,search_name):
    print(search_name[n])
    search_box = driver.find_element_by_id("searchboxinput")
    search_box.send_keys(search_name[n])
    search_box.send_keys(Keys.ENTER) #검색창에 "Los Banos 93635 Restaurants" 입력

In [30]:
pip install selenium-wire

  Using cached selenium_wire-4.6.5-py3-none-any.whl (239 kB)
  Using cached Brotli-1.0.9-cp36-cp36m-macosx_10_9_x86_64.whl (421 kB)
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install --ignore-installed certifi==2019.11.28

     |████████████████████████████████| 156 kB 10.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
from tqdm import tqdm
from seleniumwire import webdriver 
from time import sleep
import requests
import json
import pandas as pd

def google_reviews(store_list,gu,count=10):

    #드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    driver = webdriver.Chrome('/Users/minjikim/Downloads/chromedriver 4',options=options)   #chromedriver 경로 지정!
    driver.get('https://www.google.co.kr/maps')
    sleep(3)

    # 리스트 형식으로 저장된 가게 하나씩 검색
    for store in store_list:
        count=count
        driver.get('https://www.google.co.kr/maps')
        result_list=[]
        sleep(3)
        query_input=driver.find_element_by_class_name('tactile-searchbox-input')
        query_input.send_keys(gu+store+'\n')
        
        # 의도한 가게와 검색결과가 같은지 확인
        sleep(5)
        check=input('일치하는 가게를 클릭 후 y를 눌러주세요(이외 멈춤)')
        if check.upper()=='Y':
            now_url=driver.current_url
        else:
            break
        
        # 리뷰 더보기로 이동
        more_btn=driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button')
        more_btn.click()
        views=int(''.join(more_btn.text[2:-1].split(',')))//10
        
        # div태그 스크롤 
        sleep(8)
        js_scripts = '''
        let aa = document.getElementsByClassName('section-scrollbox')[0];
        setTimeout(()=>{aa.scroll(0,1000000)}, 1000);
        '''
        driver.execute_script(js_scripts)
        sleep(3) 
        
        # 헤더값 찾기
        for request in driver.requests:
            if request.response:
                pb=request.url.split('pb=')
                if len(pb)==2:
                    if pb[1][:6]=='!1m2!1':
                        url_l=request.url.split('!2m2!1i')
                        break
                        
        # json파일 들고와 리뷰 10개씩 저장하기
        # 리뷰의 수가 count보다 작을 경우 count를 리뷰의 수로 변경
        if count>views:
            for number in tqdm(range(views)):
                resp=requests.get((url_l[0]+'!2m2!1i'+'{}'+url_l[1]).format(number))
                review = json.loads(resp.text[5:])
                for user in range(10):
                    result_list.append({
                        'ID':review[2][user][0][1],
                        '내용':review[2][user][3],
                        '날짜':review[2][user][1],
                        '별점':review[2][user][4]})
        else:
            for number in tqdm(range(count)):
                resp=requests.get((url_l[0]+'!2m2!1i'+'{}'+url_l[1]).format(number))
                review = json.loads(resp.text[5:])
                for user in range(10):
                    result_list.append({
                        'ID':review[2][user][0][1],
                        '내용':review[2][user][3],
                        '날짜':review[2][user][1],
                        '별점':review[2][user][4]})
                    
        # csv로 저장
        df=pd.DataFrame(result_list)
        df.to_csv('{}.csv'.format(store))

/Users/minjikim/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509
